In [134]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_1samp, ttest_rel, linregress
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import spacy
from nltk.corpus import stopwords
import string 

from pprint import pprint 

pd.set_option('display.max_columns', 100)

In [ ]:
nlp = spacy.load('en')

In [3]:
df = pd.read_csv('./data/qwantify.csv')

In [4]:
df.head()

,startTime_ISO8601,startTime_secondsSinceMidnight1Jan1970UTC,estimatedSecondsSinceAlert,wantingAnything,whatWanting,wantingIntensity,wantedToFeel_control,wantedToFeel_lessStress,wantedToFeel_goodAboutMe,wantedToFeel_connected,wantedToFeel_comfort,wantedToFeel_novelty,wantedToFeel_calm,wantedToFeel_health,wantedToFeel_energy,wantedToFeel_goodPerson,wantedToFeel_competent,wantedToFeel_acknowledged,wantedToFeel_helpful,wantedToFeel_other,wantedToFeel_specifiedOther,wantedToFeel_nothingInParticular,doing,doing_specifiedOther,withOthers,feelingBadToGood,energy,physically_good,physically_energized,physically_hungry,physically_tired,physically_uncomfortable,physically_other,physically_specifiedOther,physically_noFeeling,feeling_angry,feelingIntensity_angry,feeling_anxious,feelingIntensity_anxious,feeling_awe,feelingIntensity_awe,feeling_content,feelingIntensity_content,feeling_frustrated,feelingIntensity_frustrated,feeling_grateful,feelingIntensity_grateful,feeling_happy,feelingIntensity_happy,feeling_jealous,feelingIntensity_jealous,feeling_loving,feelingIntensity_loving,feeling_proud,feelingIntensity_proud,feeling_restless,feelingIntensity_restless,feeling_sad,feelingIntensity_sad,feeling_other,feeling_specifiedOther,feeling_noEmotion,thinkingOverAndOver,comparingToOthers,lonely,selfWorth,appreciating,stressed
0,2017-05-02T20:11:01-07:00,1493781061,2351.0,2,Work on data project,0.494,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,talk,NaN,2,0.633,0.771,0,0,1,0,0,0,NaN,0,0,NaN,0,NaN,0,NaN,1,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.000,NaN,0.0,0.629,0.000,0.000
1,2017-05-02T12:41:30-07:00,1493754090,2354.0,2,Apply to jobs,0.281,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,0,chores,NaN,0,0.506,0.629,0,0,0,0,0,0,NaN,1,0,NaN,1,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,1,NaN,0,NaN,0,NaN,0,0.426,NaN,0.0,0.500,0.000,0.104
2,2017-05-02T08:34:34-07:00,1493739274,7.0,2,Paint the stairs,0.283,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,other,Meditating,0,0.500,0.502,0,0,0,0,1,0,NaN,0,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,1,0.175,NaN,0.0,0.363,0.000,0.289
3,2017-05-01T21:34:24-07:00,1493699664,4591.0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,other,Dancing,2,0.633,0.500,0,0,1,0,0,0,NaN,0,0,NaN,0,NaN,0,NaN,1,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.000,NaN,0.0,0.514,0.791,0.000
4,2017-05-01T15:49:41-07:00,1493678981,8616.0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,other,Looking for a job,0,0.500,0.500,0,0,0,0,0,0,NaN,1,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,1,0.000,NaN,0.0,0.502,0.000,0.143


In [14]:
df['doing'].unique()

array(['talk', 'chores', 'other', 'work', 'eat', 'travel', 'leisure',
       'sleep', 'exercise'], dtype=object)

In [15]:
df['doing_specifiedOther'].unique()

array([nan, 'Meditating', 'Dancing ', 'Looking for a job ', 'Planning',
       'At a meeting ', 'Listening to a judge', 'Walking ',
       'Writing in my journal ', 'Volunteering, gardening ',
       'Laying in sun'], dtype=object)

In [20]:
df['physically_specifiedOther'].unique()

array([nan, 'Relaxed ', 'Insomnia ', 'Weed', 'Drunk ', 'Cold'], dtype=object)

In [22]:
df['feeling_specifiedOther'].unique()

array([ nan])

In [23]:
df['whatWanting'].unique()

array(['Work on data project ', 'Apply to jobs ', 'Paint the stairs ', nan,
       'Repair stairs ', 'Clean windows ', 'Plan my weekend',
       'To present my capstone project ', 'Revise my resume ',
       'Read about machine learning ', 'Research career fair attendees ',
       'Get home and finish presentation',
       'Finish my capstone presentation ', 'Socialize with classmates',
       'Finish capstone ', 'Work on my capstone ', 'Find a good song ',
       'Go to b', 'Work on capstone ', 'Work on my capstone project ',
       'Work on project ', 'Finish my book', 'Debug my code ', 'Meditate',
       'Go outside ', 'Eat ', 'Solve this database problem ',
       'Work on capstone project ', 'Journal ',
       'Finish programming  for the day ', 'I want to eat ',
       'Exchange glasses ', 'Finish project ', 'Plan my weekend ',
       'Finish work', 'Finish lab ', 'To go to sleep ', 'Get to GA',
       'Go home ', 'Homework ', 'Get lunch ', 'Finish presentation ',
       'Work on

In [26]:
# Example 
apples, and_, oranges = nlp(u'apples and oranges')
apples.similarity(oranges)

0.77809414836023805

I think to start with I'll just implement a basic algorithm that goes through every word in the document, and compares it's corresponding word vector with a list of words (vectors) that I want to generalize to. For this example, my "master word list" will be the "doing" verbs in the qwantify app. These verbs are:
<br><br>
`['talk', 'chores', 'other', 'work', 'eat', 'travel', 'leisure','sleep', 'exercise']`

I will also add the following verbs:<br>
1. socialize 
2. relax


**Get words to compare**

In [122]:
wanting = [want for want in df['whatWanting'].unique().tolist() if str(want)!='nan']

doing = df['doing'].unique().tolist()
doing.append('socialize')
doing.append('relax')

**Pre-process words**


In [141]:
stops = set(stopwords.words("english"))

categorizations = []
for want in wanting[-4:]: # for every user-input phrase in the whatWanting field

    want = want.translate(None, string.punctuation) # remove punctuation 
    
    words = want.lower().split() 
    
    words = [nlp(w.decode('UTF-8')).text for w in words if not w in stops] # remove stop words  
    
    matches = []
    for i, word in enumerate(words): # for word in phrase
        
        word = nlp(word.decode('UTF-8')) # convert to nlp object
        
        sim = 0
        max_sim = 0
        for verb in doing: # compare each word to every word in the doing list (master list of verbs)
            
            sim = word.similarity(nlp(verb.decode('UTF-8')))
            
            if sim > max_sim:
                max_sim = sim
                best_match = verb
                print word, best_match, sim
        
        matches.append([word, best_match, max_sim]) 
#         matches.append([best_match, max_sim]) 
        
    best_match = sorted(matches, key=lambda x: x[2], reverse=True)[0]
#     best_match = sorted(matches, key=lambda x: x[1])[0]
    categorizations.append(([want] + best_match))

wash talk 0.186283703238
wash chores 0.311808610646
wash eat 0.347697972064
car talk 0.225039816635
car other 0.316571202449
car travel 0.432589722794
work talk 0.485491463295
work other 0.532549251996
work work 0.99999999063
kindle talk 0.0793085962578
kindle chores 0.13221039223
kindle other 0.133411040147
kindle work 0.151343084907
kindle travel 0.18507414611
social talk 0.392559920666
social other 0.442528870174
go talk 0.580491333037
meetup talk 0.294827764851
meetup socialize 0.402393888221
work talk 0.485491463295
work other 0.532549251996
work work 0.99999999063
kindle talk 0.0793085962578
kindle chores 0.13221039223
kindle other 0.133411040147
kindle work 0.151343084907
kindle travel 0.18507414611
parser chores 0.0186785255367
parser other 0.0415344717918
parser work 0.0628827030612


In [142]:
pprint(categorizations)

[['Wash my car', car, 'travel', 0.43258972279417607],
 ['Work on my kindle paraer', work, 'work', 0.99999999063033052],
 ['To be social  go to a meetup', go, 'talk', 0.58049133303744815],
 ['To work on kindle parser', work, 'work', 0.99999999063033052]]
